In [1]:
import gym
import procgen
import time
import numpy as np

from abc import ABC, abstractmethod

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from mpi4py import MPI
from baselines.ppo2 import ppo2
from baselines.common.models import build_impala_cnn
from baselines.common.mpi_util import setup_mpi_gpus
from procgen import ProcgenEnv
from baselines.common.vec_env import (
    VecExtractDictObs,
    VecMonitor,
    VecFrameStack,
    VecNormalize
)
from baselines import logger
from mpi4py import MPI
import argparse

from collections import deque
from baselines.common import explained_variance, set_global_seeds
from baselines.common.policies import build_policy
try:
    from mpi4py import MPI
except ImportError:
    MPI = None
from baselines.ppo2.runner import Runner

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
setup_mpi_gpus()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #pylint: disable=E1101
sess = tf.Session(config=config)
sess.__enter__()
    

In [3]:
num_envs = 64
learning_rate = 5e-4
ent_coef = .01
gamma = .999
lam = .95
nsteps = 256
nminibatches = 8
ppo_epochs = 3
clip_range = .2
use_vf_clipping = True
update_fn=None,
init_fn=None,
vf_coef=0.5,
max_grad_norm=0.5
comm = None

In [4]:
num_levels = 50
num_envs = 64
start_level = 0

# FILL IN: HOW MANY PARALLEL ENVIRONMENTS FOR TESTING
test_envs = 4

venv = ProcgenEnv(num_envs=test_envs, env_name="fruitbot", 
                  num_levels=num_levels, start_level=start_level, distribution_mode='easy')
venv = VecExtractDictObs(venv, "rgb")

venv = VecMonitor(
        venv=venv, filename=None, keep_buf=100,
    )

venv = VecNormalize(venv=venv, ob=False)


In [5]:
conv_fn = lambda x: build_impala_cnn(x, depths=[16,32,32], emb_size=256)
total_timesteps = 20000

In [6]:
load_path = None
load_path = '../train_procgen/models/sample-gpu/checkpoints/03051'


In [7]:
total_timesteps = int(total_timesteps)

policy = build_policy(venv, conv_fn)

# Get the nb of env
nenvs = venv.num_envs

# Get state_space and action_space
ob_space = venv.observation_space
ac_space = venv.action_space

# Calculate the batch_size
nbatch = nenvs * nsteps
nbatch_train = nbatch // nminibatches


# Instantiate the model object (that creates act_model and train_model)
from baselines.ppo2.model import Model
model_fn = Model

model = model_fn(policy=policy, ob_space=ob_space, ac_space=ac_space, nbatch_act=nenvs, nbatch_train=nbatch_train,
                    nsteps=nsteps, ent_coef=ent_coef, vf_coef=vf_coef,
                    max_grad_norm=max_grad_norm, comm=comm, mpi_rank_weight=0)

if load_path is not None:
        model.load(load_path)




Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [8]:
# test_envs = 1

In [9]:
venv = ProcgenEnv(num_envs=test_envs, env_name="fruitbot", 
                  num_levels=num_levels, start_level=start_level, distribution_mode='easy')
venv = VecExtractDictObs(venv, "rgb")

venv = VecMonitor(
        venv=venv, filename=None, keep_buf=100,
    )

venv = VecNormalize(venv=venv, ob=False)

In [10]:
states = model.initial_state
obs = np.zeros((test_envs,) + venv.observation_space.shape, dtype=venv.observation_space.dtype.name)
obs[:] = venv.reset()
dones = [False for _ in range(test_envs)]

In [ ]:
for steps in range(10000):
    
    # Advance environment by one timestep
    
    actions, values, states, neglogpacs = model.step(obs, S=states, M=dones)
    
    
    obs[:], rewards, dones, infos = venv.step(actions)
    
   
    venv.render()
    
    
    # time.sleep(.05)
venv.close()